In [1]:
import pymongo
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import numpy as np

In [135]:
# Connect to MongoDB
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [14]:
db = client.db_name
collection = db.items

In [10]:
# Create BeautifulSoup object for NASA website
nasa_url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
nasa_html = requests.get(nasa_url).text
nasa_soup = bs(nasa_html, 'lxml')

In [11]:
# Get first title
title_results = nasa_soup.find_all('div', class_="content_title")
title_list = []
for result in title_results:
    try:
        title = result.find('a').text.strip()
        if title:
            # print(title)
            title_list.append(title)
    except Exception as e:
        print(f'Fail: {e}')
first_title = title_list[0]
first_title

"NASA's InSight Places First Instrument on Mars"

In [12]:
# Get first paragraph
p_results = nasa_soup.find_all('div', class_="rollover_description_inner")
p_list = []
for p in p_results:
    try:
        par = p.text.strip()
        if par:
            #print(par)
            #print("--------")
            p_list.append(par)
    except Exception as e:
        print(f'Fail: {e}')
first_par = p_list[0]
first_par

'In deploying its first instrument onto the surface of Mars, the lander completes a major mission milestone.'

In [13]:
# Create splinter broswer instance
executable_path = {'executable_path':'C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe'}
browser = Browser('chrome', **executable_path)

In [183]:
# Open browser & visit website
browser.visit("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")

In [184]:
# click on "full image" button
browser.find_by_css('.button').first.click()

In [185]:
# click on the "more info" button
browser.find_by_css('.button').last.click()

In [186]:
# Click link for full image
partial_link = browser.find_by_css('.download_tiff').last.value.split(" ")[2]
browser.click_link_by_partial_href(partial_link)

In [187]:
# Get full image URL
featured_image_url = browser.url

In [16]:
# Create BeautifulSoup object for Twitter site
twitter_url = "https://twitter.com/marswxreport?lang=en"
twitter_html = requests.get(twitter_url).text
twitter_soup = bs(twitter_html, 'lxml')

In [140]:
# Get tweets
tweets = twitter_soup.find_all('div', class_ = "content")
weather_only_tweets = []

# Loop through tweets to find weather report tweets
for tweet in tweets:
    # Eliminate retweets
    username = tweet.find('span', class_ = "username u-dir u-textTruncate")
    if username.text == "@MarsWxReport":
        tweet_content = tweet.find('p', class_ = "TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text.strip()
        # Eliminate non-weather tweets
        report_test = tweet_content.split(" ")
        if report_test[0] == "Sol":
            weather_only_tweets.append(tweet_content)
mars_weather = weather_only_tweets[0]
mars_weather

'Sol 2305 (2019-01-30), high -4C/24F, low -73C/-99F, pressure at 8.14 hPa, daylight 06:47-18:54pic.twitter.com/OTkUTDyRpu'